In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyodbc

     |████████████████████████████████| 280 kB 7.9 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287361 sha256=dddb218dc022cb8cc71e69861696059ad4123b02df58fc1afdcbf7558a1f5aa4
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


In [ ]:
!pip install eurostat

     |████████████████████████████████| 45 kB 2.3 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15147 sha256=58aac344ddfcaf9e2df9dab8180202ffe6da3d5f260d0adb150edbf7c53c8c4b
  Stored in directory: /root/.cache/pip/wheels/58/88/2a/8d619cf38d7cf939e54b6ccdece05d31b64b3eb419c11d1ed3
Successfully built jsonpath-rw


In [ ]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to

In [ ]:
import gensim
import pandas as pd
import numpy as np
import re
import pyodbc

c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")
cursor = c.cursor()


def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df

#Glossary Articles
query =      """SELECT id,article_id,categories FROM ESTAT.V1.GL_articles_categories"""

glossaryArticleCategory = load_table(cursor,query)


query =      """SELECT id,definition,redirection,original_title,homepage,editorial_note,change_note,scope_note,example,history_note FROM ESTAT.V1.dat_glossary"""

glossaryArticles = load_table(cursor,query)

query = """select "ESTAT"."V1"."dat_glossary"."id", "ESTAT"."V1"."dat_glossary"."definition", "ESTAT"."V1"."dat_glossary"."redirection", "ESTAT"."V1"."dat_glossary"."original_title", "ESTAT"."V1"."dat_glossary"."homepage", "ESTAT"."V1"."dat_glossary"."editorial_note", "ESTAT"."V1"."dat_glossary"."change_note", "ESTAT"."V1"."dat_glossary"."scope_note", "ESTAT"."V1"."dat_glossary"."example", "ESTAT"."V1"."dat_glossary"."history_note", "ESTAT"."V1"."dat_link_info"."id", "ESTAT"."V1"."dat_link_info"."title", "ESTAT"."V1"."dat_link_info"."url", "ESTAT"."V1"."dat_link_info"."resource_information_id", "ESTAT"."V1"."dat_link_info"."resource_type_id" from ( "ESTAT"."V1"."dat_glossary" left outer join "ESTAT"."V1"."dat_link_info" on "ESTAT"."V1"."dat_glossary"."id" = "ESTAT"."V1"."dat_link_info"."id" )"""


glossaryArticlesTitle = load_table(cursor,query)


In [ ]:
titles = []
for name, row in glossaryArticlesTitle.iterrows():
  titles.append(str(re.sub("[^a-zA-Z\d%]", ' ', row[11]).replace("\'", "").replace("\"", "").replace("\\", "").strip()))

In [ ]:
f = open("glossaryArticles.ttl", "a")
##Map glossary articles
count = 0
for name, row in glossaryArticles.iterrows():
  f.write("estatdata:glossaryArticle" + str(row[0]) + " rdf:type estat:GlossaryArticle.\n" + "estatdata:glossaryArticle" + str(row[0]) + " estat:id \"" + str(row[0]) + "\"^^xsd:integer.\n")
  content = str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  if content != "":
    f.write("estatdata:glossaryArticle" + str(row[0]) + " rdf:type estat:GlossaryArticle.\n" + "estatdata:glossaryArticle" + str(row[0]) + " estat:content \"" + content + "\"^^xsd:string.\n")
  f.write("estatdata:glossaryArticle" + str(row[0]) + " estat:title \"" + titles[count] + "\"^^xsd:string.\n")
  count += 1

##Map glossary categories
for name, row in glossaryArticleCategory.iterrows():
  categories = row[2].replace("]", "").replace("[", "").replace("\'", "").split(", ")
  for c in categories:
    f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + " rdf:type estat:GlossaryArticleCategory.\n" + "estatdata:glossaryArticle" + str(row[1]) + " estat:hasCategoryOfGlossaryArticle estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + ".\n")
    f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
  
f.close()
  

In [ ]:
#Statistics Explained Articles
query =      """SELECT id,article_id,title,content,abstract,alert FROM ESTAT.V1.dat_article_paragraph"""

explainedArticleParagraph = load_table(cursor,query)


query =      """SELECT id,article_id,categories FROM ESTAT.V1.SE_articles_categories"""

explainedArticleCategory = load_table(cursor,query)


query =      """SELECT id,context,data_sources,background_article,homepage FROM ESTAT.V1.dat_article"""

explainedArticle = load_table(cursor,query)



In [ ]:
query = """select "ESTAT"."V1"."dat_article"."id", "ESTAT"."V1"."dat_article"."context", "ESTAT"."V1"."dat_article"."data_sources", "ESTAT"."V1"."dat_article"."background_article", "ESTAT"."V1"."dat_article"."homepage", "ESTAT"."V1"."dat_link_info"."id", "ESTAT"."V1"."dat_link_info"."title", "ESTAT"."V1"."dat_link_info"."url", "ESTAT"."V1"."dat_link_info"."resource_information_id", "ESTAT"."V1"."dat_link_info"."resource_type_id" from ( "ESTAT"."V1"."dat_article" left outer join "ESTAT"."V1"."dat_link_info" on "ESTAT"."V1"."dat_article"."id" = "ESTAT"."V1"."dat_link_info"."id" )"""
explainedArticleTitles = load_table(cursor,query)

In [ ]:
context, data_source, title = [], [], []
for name, row in explainedArticleTitles.iterrows():
  try:
    context.append(str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip()))
  except Exception as e:
    context.append("None")
  try:
    data_source.append(str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()))
  except Exception as e:
    data_source.append("None")
  try:
    title.append(str(re.sub("[^a-zA-Z\d%]", ' ', row[6]).replace("\'", "").replace("\"", "").replace("\\", "").strip()))
  except Exception as e:
    title.append("None")

In [ ]:
#Map statistical article
count = 0
f = open("explainedArticles.ttl", "a")
for name, row in explainedArticle.iterrows():
  helperExplained = ""
  if row[3] == 0:
    try:
        f.write("estatdata:SEArticle" + str(row[0]) + " rdf:type estat:StatisticsExplainedArticle.\n")
        f.write("estatdata:SEArticle" + str(row[0]) + " estat:id \"" + str(row[0]) + "\"^^xsd:integer.\n")
        if context[count] != "":
          f.write("estatdata:SEArticle" + str(row[0]) + " estat:context \"" + context[count] + "\"^^xsd:string.\n")
        if data_source[count] != "":
          f.write("estatdata:SEArticle" + str(row[0]) + " estat:dataSource \"" + data_source[count] + "\"^^xsd:string.\n")
        f.write("estatdata:SEArticle" + str(row[0]) + " estat:title \"" + str(title[count]) + "\"^^xsd:string.\n")
    except Exception as e:
      print("a")
  else:
    try:
      f.write("estatdata:SEArticle" + str(row[0]) + " rdf:type estat:BackgroundArticle.\n")
      f.write("estatdata:SEArticle" + str(row[0]) + " estat:id \"" + str(row[0]) + "\"^^xsd:integer.\n")        
      if context[count] != "":
        f.write("estatdata:SEArticle" + str(row[0]) + " estat:context \"" + context[count] + "\"^^xsd:string.\n")
      if data_source[count] != "":
        f.write("estatdata:SEArticle" + str(row[0]) + " estat:dataSource \"" + data_source[count] + "\"^^xsd:string.\n")
      f.write("estatdata:SEArticle" + str(row[0]) + " estat:title \"" + str(title[count]) + "\"^^xsd:string.\n")
    except Exception as e:
      print("a")
  count += 1
f.close()

In [ ]:
#Map statistical article categories
f = open("explainedArticles.ttl", "a")
for name, row in explainedArticleCategory.iterrows():
  categories = row[2].replace("]", "").replace("[", "").replace("\'", "").split(", ")
  for c in categories:
    f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + " rdf:type estat:StatisticsExplainedCategory.\nestatdata:SEArticle" + str(row[1]) + " estat:hasCategoryOfStatisticExplainedArticle estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + ".\n")
    f.write("estatdata:" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip().replace(" ", "_")).lower() + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', c).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
      
f.close()

In [ ]:
#Map statistical article paragraphs
f = open("explainedArticles.ttl", "a")
for name, row in explainedArticleParagraph.iterrows():
  f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasParagraph estatdata:paragraph" + str(row[1]) + "_" + str(row[0]) + ".\nestatdata:paragraph" + str(row[1]) + "_" + str(row[0]) + " rdf:type estat:Paragraph.\n")
  f.write("estatdata:paragraph" + str(row[1]) + "_" + str(row[0]) + " estat:title \"" + str(re.sub("[^a-zA-Z\d%]", ' ', row[2]).replace("\'", "").replace("\"", "").replace("\\", "").strip()) + "\"^^xsd:string.\n")
  content = str(re.sub("[^a-zA-Z\d%]", ' ', row[3]).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  if content != "":
    f.write("estatdata:paragraph" + str(row[1]) + "_" + str(row[0]) + " estat:content \"" + content + "\"^^xsd:string.\n")


f.close()